<a href="https://colab.research.google.com/github/ogdura/dinamic/blob/main/Lb4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1> Лбораторная работа 4</h2>


Загружаем библиотеки

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.naive_bayes
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

Подготавливаем данные: определяем признаки и таргет, разделяем признаки на категориальные и численные и приобразуем их для обучения


In [34]:
df = pd.read_csv("fraud.csv",sep=',')

features = ['trans_date_trans_time', 'amt', 'category', 'merchant', 'city', 'state', 'job', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'gender']
target = 'is_fraud'

fraud=df[df[target]==1]
not_fraud=df[df[target]==0]
not_fraud_down_sample=not_fraud.sample(n=11000)
under=pd.concat([fraud,not_fraud_down_sample])

x = under[features]
y = under[target]

print((x['amt']<0).any())


x['hour']=pd.to_datetime(x['trans_date_trans_time'],dayfirst=True).dt.hour
x=x.drop(['trans_date_trans_time'],axis=1)
categorical_features = ['category', 'merchant', 'city', 'state', 'job', 'gender']
numeric_features = ['amt', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'hour']

# Преобразование координат (сдвиг значений, чтобы все были неотрицательными)
x['lat'] = x['lat'] - x['lat'].min()
x['long'] = x['long'] - x['long'].min()
x['merch_lat'] = x['merch_lat'] - x['merch_lat'].min()
x['merch_long'] = x['merch_long'] - x['merch_long'].min()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


x = preprocessor.fit_transform(x)

False


<ipython-input-34-f715a8ecf2da>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['hour']=pd.to_datetime(x['trans_date_trans_time'],dayfirst=True).dt.hour


In [44]:
#smote=SMOTE()
#x,y=smote.fit_resample(x,y)
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.2)
x_train=x_train.toarray()
x_test=x_test.toarray()

<h2>Гауссовский наивный Байес</h2>
Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметра "var_smoothing", который отвечает за сглаживание получаемых значений.

In [ ]:
gaussian_classifier = sklearn.naive_bayes.GaussianNB()
gaussian_params ={
    'var_smoothing': np.geomspace(1e-10,1e10,num=100)
}
gaussian_grid = GridSearchCV(gaussian_classifier, gaussian_params, cv=2, n_jobs=-1,verbose=3)
gaussian_grid.fit(x_train, y_train)
best_gaussian_params = gaussian_grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [ ]:
best_gaussian_model = sklearn.naive_bayes.GaussianNB(**best_gaussian_params)
best_gaussian_model.fit(x_train, y_train)
gaussian_predicted = best_gaussian_model.predict(x_test)
print(best_gaussian_params)
print(metrics.classification_report(y_test, gaussian_predicted,digits=5))

{'var_smoothing': 0.12328467394420634}
              precision    recall  f1-score   support

           0    0.95361   0.98533   0.96921     11997
           1    0.86667   0.66550   0.75288      1719

    accuracy                        0.94525     13716
   macro avg    0.91014   0.82542   0.86105     13716
weighted avg    0.94272   0.94525   0.94210     13716



# <h2> Мультиномиальный наивный Байес </h2>
Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметра "alpha", который отвечает за сглаживание получаемых значений.

In [35]:
classifier = sklearn.naive_bayes.MultinomialNB()
params ={
    'alpha': np.geomspace(1e-100,1e100,num=100)
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(x_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [45]:
best_model = sklearn.naive_bayes.MultinomialNB(**best_params)
best_model.fit(x_train, y_train)
predicted = best_model.predict(x_test)
print(best_params)
print(metrics.classification_report(y_test, predicted,digits=5))

{'alpha': 10.235310218990268}
              precision    recall  f1-score   support

           0    0.84152   0.72540   0.77916      8740
           1    0.76157   0.86524   0.81010      8860

    accuracy                        0.79580     17600
   macro avg    0.80155   0.79532   0.79463     17600
weighted avg    0.80127   0.79580   0.79474     17600



<h2>Комплементарный наивный Байес</h2>
Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметра "alpha", который отвечает за сглаживание получаемых значений.

In [46]:
classifier = sklearn.naive_bayes.ComplementNB()
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(x_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [47]:
best_model = sklearn.naive_bayes.ComplementNB(**best_params)
best_model.fit(x_train, y_train)
predicted = best_model.predict(x_test)
print(best_params)
print(metrics.classification_report(y_test, predicted,digits=5))

{'alpha': 0.007564633275546291}
              precision    recall  f1-score   support

           0    0.91237   0.84222   0.87589      8740
           1    0.85533   0.92020   0.88658      8860

    accuracy                        0.88148     17600
   macro avg    0.88385   0.88121   0.88124     17600
weighted avg    0.88366   0.88148   0.88127     17600



<h2>Бернуллиевский наивный Байес</h2>
Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметра "alpha", который отвечает за сглаживание получаемых значений.

In [48]:
classifier = sklearn.naive_bayes.BernoulliNB()
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(x_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [50]:
best_model = sklearn.naive_bayes.BernoulliNB(**best_params)
best_model.fit(x_train, y_train)
predicted = best_model.predict(x_test)
print(best_params)
print(metrics.classification_report(y_test, predicted,digits=5))

{'alpha': 0.019179102616724848}
              precision    recall  f1-score   support

           0    0.90807   0.85217   0.87924      8740
           1    0.86252   0.91490   0.88794      8860

    accuracy                        0.88375     17600
   macro avg    0.88530   0.88354   0.88359     17600
weighted avg    0.88514   0.88375   0.88362     17600



<h3> Категориальный наивный Байес </h3>Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметра "alpha", который отвечает за сглаживание получаемых значений.

In [51]:
classifier = sklearn.naive_bayes.CategoricalNB()
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100),
    'min_categories':[2]
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(x_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.



In [54]:
best_model = sklearn.naive_bayes.CategoricalNB(**best_params)
best_model.fit(x_train, y_train)
predicted = best_model.predict(x_test)
print(best_params)
print(metrics.classification_report(y_test, predicted,digits=5))

{'alpha': 0.012045035402587835, 'min_categories': 2}
              precision    recall  f1-score   support

           0    0.92506   0.89828   0.91148      8740
           1    0.90245   0.92822   0.91515      8860

    accuracy                        0.91335     17600
   macro avg    0.91375   0.91325   0.91331     17600
weighted avg    0.91368   0.91335   0.91333     17600

